In [16]:
import cv2 as cv
import numpy as np

### Wagner

In [5]:
# Inicializa o vídeo e o algoritmo de subtração de fundo
cap = cv.VideoCapture('../src/barcodes.avi')

while True:
    ret, frame = cap.read()
    
    # Verifica se o frame foi lido corretamente
    if frame is None:
        break
    # frame = cv.resize(frame, (0,0), fx=0.5, fy=0.5)
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    grad_x = cv.Sobel(gray, cv.CV_16S, 1, 0)
    grad_y = cv.Sobel(gray, cv.CV_16S, 0, 1)
    grad = cv.subtract(grad_x, grad_y)
    grad = cv.convertScaleAbs(grad)

    grad_x = cv.medianBlur(grad_x, 3)
    _, grad_x = cv.threshold(grad_x, 50, 255, cv.THRESH_BINARY)
    kernel = np.ones((7, 7), np.uint8)
    #kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3), (-1, -1))

    grad_x = cv.morphologyEx(grad_x, cv.MORPH_CLOSE, kernel)
    grad_x = cv.morphologyEx(grad_x, cv.MORPH_OPEN, kernel)

    # Exibe a máscara do primeiro plano

    grad_x = cv.convertScaleAbs(grad_x)
    grad_y= cv.convertScaleAbs(grad_y)

    cv.imshow('Image', grad_x)
    cv.imshow('Image_', frame)

    # Sai do loop ao pressionar 'q'
    if cv.waitKey(30) == ord('q'):
        break

# Libera os recursos e fecha as janelas
cap.release()
cv.destroyAllWindows()


### Kaio

In [16]:
cap = cv.VideoCapture('../src/barcodes.avi')
cv.namedWindow('Códigos de Barra Detectados', cv.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    gradX = cv.Sobel(gray, ddepth=cv.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv.Sobel(gray, ddepth=cv.CV_32F, dx=0, dy=1, ksize=-1)

    gradient = cv.subtract(gradX, gradY)
    gradient = cv.convertScaleAbs(gradient)

    blurred = cv.blur(gradient, (9, 9))
    (_, thresh) = cv.threshold(blurred, 225, 255, cv.THRESH_BINARY)

    # Operações morfológicas para fechar buracos
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (21, 7))
    closed = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)

    closed = cv.erode(closed, None, iterations=4)
    closed = cv.dilate(closed, None, iterations=4)

    cnts, _ = cv.findContours(closed.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if cnts:
        # Pegar o maior contorno
        c = sorted(cnts, key=cv.contourArea, reverse=True)[0]

        # Calcular o retângulo rotacionado de menor área
        rect = cv.minAreaRect(c)

        # Obter os pontos da caixa rotacionada
        box = cv.boxPoints(rect)
        box = np.int0(box)

        # Desenhar o retângulo preenchido ao redor do código de barras
        if rect[1][0] > rect[1][1]:  # Verificar se o código de barras é horizontal
            cv.fillConvexPoly(frame, box, (255, 0, 0))  # Azul para horizontal
        else:  # Se é vertical
            cv.fillConvexPoly(frame, box, (0, 0, 255))  # Vermelho para vertical

    cv.imshow('Códigos de Barra Detectados', frame)

    if cv.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


In [18]:
import cv2 as cv
import numpy as np

# Carregar o vídeo
cap = cv.VideoCapture('../src/barcodes.avi')
cv.namedWindow('Códigos de Barra Detectados', cv.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    gradX = cv.Sobel(gray, ddepth=cv.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv.Sobel(gray, ddepth=cv.CV_32F, dx=0, dy=1, ksize=-1)

    gradient = cv.subtract(gradX, gradY)
    gradient = cv.convertScaleAbs(gradient)

    blurred = cv.blur(gradient, (9, 9))
    (_, thresh) = cv.threshold(blurred, 225, 255, cv.THRESH_BINARY)

    # Operações morfológicas para fechar buracos
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (21, 7))
    closed = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)

    closed = cv.erode(closed, None, iterations=4)
    closed = cv.dilate(closed, None, iterations=4)

    cnts, _ = cv.findContours(closed.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if cnts:
        # Pegar o maior contorno
        c = sorted(cnts, key=cv.contourArea, reverse=True)[0]

        x, y, w, h = cv.boundingRect(c)

        if w > h:  # Horizontal
            cv.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), -1)  # Azul para horizontal
        else:  # Vertical
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), -1)  # Vermelho para vertical

    cv.imshow('Códigos de Barra Detectados', frame)

    if cv.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
